# HOMEWORK 2: NUMPY FOR DATA SCIENCE

Họ tên: Lê Hà Thanh Chương

MSSV: 23120195

---

## Khám phá dữ liệu

In [1]:
cols = {}
file_path = "../data/raw/aug_train.csv"

# Read the aug_test.csv file
with open(file_path, "r", encoding="utf-8") as f:
    header = f.readline().rstrip("\n").split(",")
    for col in header:
        cols[col] = []
    num_cols = len(header)
    for line in f:
        parts = line.rstrip("\n").split(",")
        if len(parts) < num_cols:
            parts += [""] * (num_cols - len(parts))
        elif len(parts) > num_cols:
            parts = parts[:num_cols]
        for col, val in zip(header, parts):
            cols[col].append(val)

### Dữ liệu có bao nhiêu dòng và bao nhiêu cột?

In [2]:
# Các trường của dữ liệu
cols.keys()

dict_keys(['enrollee_id', 'city', 'city_development_index', 'gender', 'relevent_experience', 'enrolled_university', 'education_level', 'major_discipline', 'experience', 'company_size', 'company_type', 'last_new_job', 'training_hours', 'target'])

In [3]:
# Số dòng
num_rows = len(next(iter(cols.values())))   
print(f"Rows: {num_rows}")

Rows: 19158


In [4]:
# Số cột
num_cols = len(cols)
print(f"Columns: {num_cols}")

Columns: 14


### Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

Mỗi dòng biểu diễn một bản ghi độc lập về một ứng viên (enrollee), chứa thông tin tổng hợp về cá nhân đó.

Kiểm tra cấu trúc dữ liệu cho thấy tất cả các dòng đều tuân theo một định dạng thống nhất, không xuất hiện sự pha trộn giữa các loại thực thể. Điều này cho thấy dữ liệu đồng nhất và đảm bảo rằng mỗi dòng mang cùng một ngữ nghĩa: mô tả đặc điểm của một ứng viên tại một thời điểm.

### Dữ liệu có các dòng bị lặp không?

In [5]:
ids = cols["enrollee_id"]
unique_ids = set(ids)
duplicated_id = len(unique_ids) != len(ids)
duplicated_id

False

Tiếp tục kiểm tra tính toàn vẹn dữ liệu, việc đối chiếu giá trị trùng lặp dựa trên khóa nhận dạng enrollee_id cho kết quả không có dòng bị lặp, cho thấy bộ dữ liệu không tồn tại các bản ghi dư thừa.

### Mỗi cột có ý nghĩa gì?

Bộ dữ liệu gồm 14 biến đầu vào, phản ánh toàn diện các khía cạnh liên quan đến ứng viên trong thị trường tuyển dụng ngành Data Science. Dựa trên mô tả từ nguồn dữ liệu và việc quan sát trực tiếp các giá trị mẫu, các biến có thể được phân loại và giải thích như sau:

- enrollee_id: id định danh duy nhất của mỗi ứng viên.
- city: Mã định danh của thành phố nơi ứng viên sinh sống hoặc làm việc.
- city_development_index là biến liên tục (continuous) trong khoảng [0, 1], thể hiện mức độ phát triển của thành phố – yếu tố có thể ảnh hưởng đáng kể đến cơ hội việc làm và khả năng dịch chuyển nghề nghiệp.
- gender: Giới tính của ứng viên.
- relevent_experience: Ứng viên có kinh nghiệm liên quan đến lĩnh vực Data Science hay không.
- enrolled_university: Tình trạng học tập hiện tại.
- education_level: Trình độ học vấn cao nhất.
- major_discipline: Ngành học chính.
- experience: Số năm kinh nghiệm tổng cộng.
- company_size: Quy mô công ty hiện tại.
- company_type: Loại hình tổ chức công ty nơi ứng viên đang làm việc, nhưng chứa tỷ lệ missing tương đối lớn.
- last_new_job: Số năm kể từ công việc mới gần nhất.
- training_hours: Số giờ đào tạo mà ứng viên đã hoàn thành.
- target: Cho biết nhân viên có nghỉ việc hay không

### Mỗi cột hiện đang có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp không?

In [6]:
for col_name, col_vals in cols.items():
    print(f'{col_name}: {type(col_vals[0])}')

enrollee_id: <class 'str'>
city: <class 'str'>
city_development_index: <class 'str'>
gender: <class 'str'>
relevent_experience: <class 'str'>
enrolled_university: <class 'str'>
education_level: <class 'str'>
major_discipline: <class 'str'>
experience: <class 'str'>
company_size: <class 'str'>
company_type: <class 'str'>
last_new_job: <class 'str'>
training_hours: <class 'str'>
target: <class 'str'>


Đúng như dự đoán, tất cả các cột đều đang có kiểu dữ liệu là str. Để có thể tiếp tục khám phá tiếp thì ta cần tiền xử lý để các cột có kiểu dữ liệu phù hợp.

Cụ thể, các biến phân loại (*city*, *gender*, *relevent_experience*, *enrolled_university*, *education_level*, *major_discipline*, *company_type*) sẽ được giữ nguyên ở dạng chuỗi vì phù hợp với bản chất của chúng. Tuy nhiên những biến này sau đó có thể được xử lý bằng các kỹ thuật mã hóa phù hợp (one-hot encoding, target encoding…) ở các bước kế tiếp.

Ngược lại, nhóm biến có bản chất định lượng hoặc thứ tự cần được chuyển đổi về dạng số. Biến *city_development_index* sẽ được chuyển đổi sang kiểu `float` để phản ánh đúng giá trị liên tục trong khoảng [0, 1]. Biến *training_hours* được chuyển sang dạng `float` (mặc dù bản chất dữ liệu là số nguyên) nhằm hỗ trợ xử lý các giá trị missing, phân tích thống kê và mô hình hóa.

Các biến thứ tự như *experience*, *last_new_job*, và *company_size* cần được ánh xạ về giá trị số theo quy tắc nhất quán. Ví dụ, “<1” được quy đổi thành 0, “>20” thành 21 đối với *experience*; giá trị “never” được ánh xạ thành 0 và “>4” thành nhóm cao nhất đối với *last_new_job*; trong khi *company_size* sẽ được mã hóa thành thang đo thứ tự từ quy mô nhỏ nhất đến lớn nhất. 


---